In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.append('../') 
from run_utils import get_env_dataset, run_env_experiment, plot_ratings_mses, s3_experiment_dir_name
from env_defaults import TOPICS_STATIC, get_len_trial
from reclab.environments import Topics

from reclab.recommenders import LibFM
from reclab.recommenders import KNNRecommender
from reclab.recommenders import TopPop


/Users/wenshuoguo/Documents/GitHub/recsys-eval/experiments/run_utils.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import tqdm.autonotebook
Using TensorFlow backend.


# Behavior with user preference shift

In [2]:
## Key Parameters across all settings
bucket_name = 'recsys-eval'
data_dir = 'Wenshuo_dynamic'
overwrite = True

# Experiment setup.
num_users = TOPICS_STATIC['params']['num_users']
num_init_ratings = TOPICS_STATIC['optional_params']['num_init_ratings']
num_final_ratings = TOPICS_STATIC['misc']['num_final_ratings']
rating_frequency = TOPICS_STATIC['optional_params']['rating_frequency']

n_trials = 11
len_trial = get_len_trial(TOPICS_STATIC)
trial_seeds = [i for i in range(n_trials)]

# Environment setup
environment_name = TOPICS_STATIC['name']
env = Topics(**TOPICS_STATIC['params'], **TOPICS_STATIC['optional_params'])

# Recommender setup
recommender_name = 'LibFM (SGD)'
recommender_class = LibFM

In [3]:
# ====Step 5====
starting_data = get_env_dataset(env)

In [4]:
# Set parameters based on tuning
num_two_way_factors = 20
num_iter = 200
init_stdev = 1.0
reg = 0.01
lr = 0.001

In [5]:
default_params = dict(num_user_features=0,
                      num_item_features=0,
                      num_rating_features=0,
                      max_num_users=num_users,
                      max_num_items=TOPICS_STATIC['params']['num_items'],
                      method='sgd')

recommender = recommender_class(num_iter=num_iter,
                                num_two_way_factors=num_two_way_factors,
                                init_stdev=init_stdev,
                                reg=reg,
                                learning_rate=lr,
                                **default_params)

In [ ]:
for i, seed in enumerate(trial_seeds):
    run_env_experiment(
            [env],
            [recommender],
            [seed],
            len_trial,
            environment_names=[environment_name],
            recommender_names=[recommender_name],
            bucket_name=bucket_name,
            data_dir=data_dir,
            overwrite=overwrite, shift=True, shift_step=10)

Started experiments on environment: topics_static
	Initial density: 5.88235294117647%, Final density: 11.76470588235294%, Good item density: 29.654588235294117%
Running trials for recommender: LibFM (SGD)
Running trial with seed: 0


Applying preference shift at step  10


In [ ]:
# Recommender setup
recommender_name = 'ItemKnn'
recommender_class = KNNRecommender

In [ ]:
neighborhood_size = 250
shrinkage = 0
recommender = recommender_class(shrinkage=shrinkage, neighborhood_size=neighborhood_size)
for i, seed in enumerate(trial_seeds):
    run_env_experiment(
            [env],
            [recommender],
            [seed],
            len_trial,
            environment_names=[environment_name],
            recommender_names=[recommender_name],
            bucket_name=bucket_name,
            data_dir=data_dir,
            overwrite=overwrite,shift=True, shift_step=10)

In [ ]:
from reclab.recommenders import PerfectRec

# Recommender setup
recommender_name = 'PerfectRec'
recommender_class = PerfectRec

def rating_func():
    ratings = env.dense_ratings
    return ratings

recommender = recommender_class(rating_func)
for i, seed in enumerate(trial_seeds):
    run_env_experiment(
            [env],
            [recommender],
            [seed],
            len_trial,
            environment_names=[environment_name],
            recommender_names=[recommender_name],
            bucket_name=bucket_name,
            data_dir=data_dir,
            overwrite=overwrite, shift=True, shift_step=10)

In [ ]:

# Recommender setup
recommender_name = 'TopPop'
recommender_class = TopPop
recommender = recommender_class()

for i, seed in enumerate(trial_seeds):
    run_env_experiment(
            [env],
            [recommender],
            [seed],
            len_trial,
            environment_names=[environment_name],
            recommender_names=[recommender_name],
            bucket_name=bucket_name,
            data_dir=data_dir,
            overwrite=overwrite, shift=True, shift_step=10)